In [1]:
%config IPCompleter.use_jedi = False

### Import Required Libraries

In [2]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

### Reading a Music File (midi format)

In [3]:
midi = converter.parse('piano_tunes/BlueStone_LastDungeon.mid')

In [4]:
midi

<music21.stream.Score 0x13305057f88>

In [5]:
midi.show('midi') # For listening to songs

In [6]:
midi.show('text') # Whole music in text format

{0.0} <music21.stream.Part 0x13305061c88>
    {0.0} <music21.instrument.Piano 'Right Hand: Piano'>
    {0.0} <music21.instrument.Piano 'Right Hand: Piano'>
    {0.0} <music21.tempo.MetronomeMark Quarter=124.0>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.stream.Voice 0x132fe691f88>
        {0.0} <music21.chord.Chord C#4 E4 A4>
        {7.75} <music21.note.Rest rest>
        {8.0} <music21.chord.Chord D4 F4 B-4>
        {15.75} <music21.note.Rest rest>
        {16.0} <music21.chord.Chord E-4 F#4 B4>
        {23.75} <music21.note.Rest rest>
        {24.0} <music21.chord.Chord E4 G4 C5>
        {31.75} <music21.note.Rest rest>
        {32.0} <music21.chord.Chord C#4 E4 A4>
        {35.75} <music21.note.Rest rest>
        {39.0} <music21.chord.Chord C4 E4 A4>
        {39.3333} <music21.chord.Chord C4 E4 A4>
        {39.6667} <music21.chord.Chord C4 E4 A4>
        {40.0} <music21.chord.Chord B3 D4 G4>
        {41.75} <music21.note.Rest rest>
        {42.0} <music21.chord.C

In [7]:
elements_to_parse = midi.flat.notes # Returns an iterator

In [8]:
for e in elements_to_parse:
    print(e, e.offset)
    # Offset tells that at which moment the element will be played

<music21.chord.Chord C#4 E4 A4> 0.0
<music21.note.Note A> 0.0
<music21.note.Note A> 1.5
<music21.note.Note A> 2.5
<music21.note.Note A> 3.5
<music21.note.Note A> 4.0
<music21.note.Note A> 5.5
<music21.note.Note A> 6.5
<music21.note.Note A> 7.0
<music21.chord.Chord D4 F4 B-4> 8.0
<music21.note.Note A> 8.0
<music21.note.Note A> 9.5
<music21.note.Note A> 10.5
<music21.note.Note A> 11.5
<music21.note.Note A> 12.0
<music21.note.Note A> 13.5
<music21.note.Note A> 14.5
<music21.note.Note A> 15.0
<music21.chord.Chord E-4 F#4 B4> 16.0
<music21.note.Note A> 16.0
<music21.note.Note A> 17.5
<music21.note.Note A> 18.5
<music21.note.Note A> 19.5
<music21.note.Note A> 20.0
<music21.note.Note A> 21.5
<music21.note.Note A> 22.5
<music21.note.Note A> 23.0
<music21.chord.Chord E4 G4 C5> 24.0
<music21.note.Note A> 24.0
<music21.note.Note A> 25.5
<music21.note.Note A> 26.5
<music21.note.Note A> 27.5
<music21.note.Note A> 28.0
<music21.note.Note A> 29.5
<music21.note.Note A> 30.5
<music21.note.Note A> 31.0


<music21.chord.Chord C4 E4 A4> 304.0
<music21.note.Note A> 304.0
<music21.note.Note A> 306.0
<music21.note.Note A> 308.0
<music21.note.Note A> 309.0
<music21.note.Note A> 310.0


In [9]:
# Note
print(isinstance(elements_to_parse[3],note.Note))
print(isinstance(elements_to_parse[3],chord.Chord))
print(str(elements_to_parse[3].pitch))

True
False
A2


In [10]:
# Chord
print(isinstance(elements_to_parse[91],note.Note))
print(isinstance(elements_to_parse[91],chord.Chord))
print(elements_to_parse[91].normalOrder)
print("+".join(str(n) for n in elements_to_parse[91].normalOrder))

False
True
[9, 0, 4]
9+0+4


In [11]:
notes_demo = []

for ele in elements_to_parse:
    # if ele is a Note, then store its pitch
    if isinstance(ele,note.Note):
        notes_demo.append(str(ele.pitch))
    
    # if ele is a Chord, then split each node of chord and join them with +
    elif isinstance(ele,chord.Chord):
        notes_demo.append("+".join(str(n) for n in ele.normalOrder))

In [12]:
len(notes_demo)

682

### Preprocessing all files

In [13]:
notes = []

for file in glob.glob("piano_tunes/*.mid"):
    midi = converter.parse(file) # Converts file into stream.Score object
    
    print("Parsing %s"%file)
    
    elements_to_parse = midi.flat.notes
    
    for ele in elements_to_parse:
        # if ele is a Note, then store its pitch
        if isinstance(ele,note.Note):
            notes.append(str(ele.pitch))

        # if ele is a Chord, then split each node of chord and join them with +
        elif isinstance(ele,chord.Chord):
            notes.append("+".join(str(n) for n in ele.normalOrder))

Parsing piano_tunes\0fithos.mid
Parsing piano_tunes\8.mid
Parsing piano_tunes\ahead_on_our_way_piano.mid
Parsing piano_tunes\AT.mid
Parsing piano_tunes\balamb.mid
Parsing piano_tunes\bcm.mid
Parsing piano_tunes\BlueStone_LastDungeon.mid
Parsing piano_tunes\braska.mid
Parsing piano_tunes\caitsith.mid
Parsing piano_tunes\Cids.mid
Parsing piano_tunes\cosmo.mid
Parsing piano_tunes\costadsol.mid
Parsing piano_tunes\dayafter.mid
Parsing piano_tunes\decisive.mid
Parsing piano_tunes\dontbeafraid.mid
Parsing piano_tunes\DOS.mid
Parsing piano_tunes\electric_de_chocobo.mid
Parsing piano_tunes\Eternal_Harvest.mid
Parsing piano_tunes\EyesOnMePiano.mid
Parsing piano_tunes\ff11_awakening_piano.mid
Parsing piano_tunes\ff1battp.mid
Parsing piano_tunes\FF3_Battle_(Piano).mid
Parsing piano_tunes\FF3_Third_Phase_Final_(Piano).mid
Parsing piano_tunes\ff4-airship.mid
Parsing piano_tunes\Ff4-BattleLust.mid
Parsing piano_tunes\ff4-fight1.mid
Parsing piano_tunes\ff4-town.mid
Parsing piano_tunes\FF4.mid
Parsing

In [14]:
len(notes)

60866

In [15]:
with open("notes.pkl","wb") as filepath:
    pickle.dump(notes, filepath)

In [16]:
with open("notes.pkl","rb") as filepath:
    notes = pickle.load(filepath)

In [17]:
n_vocab = len(set(notes)) # Unique elements

In [18]:
print("Total Notes -",len(notes))
print("Unique Notes -",n_vocab)

Total Notes - 60866
Unique Notes - 359


In [19]:
print(notes[:100])

['4+9', 'E2', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', '11+4', '4+9', '11+4', '4+9', '4+9', '4+9', '4+9', '4+9', '0+4', 'E2', '4+9', '0+4', '4+9', '4+9', '4+9', '4+9', '4+9', '9+2', '4+9', '9+2', '9+2', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', 'E2', '4+9', '4+9', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '5+11', '4+9', '5+11', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '9+0', 'E2', '4+9', '9+0', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '11+2', '4+9', '11+2', '11+2', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '3+7+11', 'E-2', '3+7+11', 'B2', 'G2', '1+5+9', 'F#2', '1+5+9', '3+7+11', 'E-2', '3+7+11', 'G2', 'B2', 'E-3']


### Prepare Sequential Data for LSTM

In [20]:
# How many elements LSTM input should consider
sequence_length = 100

In [21]:
pitchnames = sorted(set(notes))

In [22]:
# Mapping between ele and integers
ele_2_int = dict((ele,num) for num,ele in enumerate(pitchnames))

In [23]:
len(ele_2_int)

359

In [24]:
network_input = []
network_output = []

In [25]:
for i in range(len(notes)-sequence_length):
    seq_in = notes[i:i+sequence_length] # Contains 100 values
    seq_out = notes[i+sequence_length]
    
    network_input.append([ele_2_int[ele] for ele in seq_in])
    network_output.append(ele_2_int[seq_out])

In [26]:
n_patterns = len(network_input)
print(n_patterns)

60766


In [27]:
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
print(network_input.shape)

(60766, 100, 1)


In [28]:
normalised_network_input = network_input/float(n_vocab)

In [29]:
# network output are the classes, encode into one hot vector
network_output = to_categorical(network_output)

In [30]:
network_output.shape

(60766, 359)

### Create Model

In [31]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [32]:
model = Sequential()
model.add(LSTM(units=512,input_shape=(normalised_network_input.shape[1],normalised_network_input.shape[2]),
               return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(512,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab,activation='softmax'))

In [33]:
model.compile(loss="categorical_crossentropy",optimizer='adam')

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 512)          1052672   
_________________________________________________________________
dropout (Dropout)            (None, 100, 512)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

In [35]:
checkpoint = ModelCheckpoint("model.hdf5", monitor="loss", verbose=0, save_best_only=True, mode='min')

hist = model.fit(normalised_network_input, network_output,
                epochs=100,
                batch_size=64,
                callbacks=[checkpoint,]
                )

Epoch 1/100
950/950 [==============================] - 143s 146ms/step - loss: 4.7690
Epoch 2/100
950/950 [==============================] - 142s 149ms/step - loss: 4.7198
Epoch 3/100
950/950 [==============================] - 142s 150ms/step - loss: 4.7146
Epoch 4/100
950/950 [==============================] - 143s 150ms/step - loss: 4.7127
Epoch 5/100
950/950 [==============================] - 141s 148ms/step - loss: 4.7113
Epoch 6/100
950/950 [==============================] - 141s 148ms/step - loss: 4.7098
Epoch 7/100
950/950 [==============================] - 143s 151ms/step - loss: 4.7096
Epoch 8/100
950/950 [==============================] - 145s 153ms/step - loss: 4.7084
Epoch 9/100
950/950 [==============================] - 146s 153ms/step - loss: 4.7079
Epoch 10/100
950/950 [==============================] - 141s 148ms/step - loss: 4.7075
Epoch 11/100
950/950 [==============================] - 141s 148ms/step - loss: 4.7077
Epoch 12/100
950/950 [==============================

950/950 [==============================] - 141s 148ms/step - loss: 4.6999
Epoch 96/100
950/950 [==============================] - 139s 147ms/step - loss: 4.7002
Epoch 97/100
950/950 [==============================] - 145s 152ms/step - loss: 4.7003
Epoch 98/100
950/950 [==============================] - 142s 149ms/step - loss: 4.6997
Epoch 99/100
950/950 [==============================] - 141s 148ms/step - loss: 4.7002
Epoch 100/100
950/950 [==============================] - 141s 149ms/step - loss: 4.7001


In [ ]:
model = load_model("model.hdf5")